In [ ]:
import os
import glob
import rasterio
from rasterio.plot import show, show_hist
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import csv

import warnings
warnings.filterwarnings("ignore")

year = 2020

filserver = "/filserver/frrov/imagery"

s1_folder = os.path.join(filserver, "Sentinel-1")
s2_folder = os.path.join(filserver, "Sentinel-2")

s1_imgs = glob.glob(f"{s1_folder}/{year}/*_aligned.tiff")
s2_imgs = glob.glob(f"{s2_folder}/{year}/*_aligned.tiff")

In [ ]:
# def pct_clip(array, pct=[2,98]):
#         array_min, array_max = np.nanpercentile(array, pct[0]), np.nanpercentile(array, pct[1])
#         clip = (array - array_min) / (array_max - array_min)
#         clip[clip>1] = 1
#         clip[clip<0] = 0
#         return clip

# def show_single_band_img(path, band=1, pct=[0,100], hist=False, meta=False):

#     raster = rasterio.open(path)
#     if meta:
#         print(raster.meta)
#     if hist:
#         show_hist(source=raster, bins=50, title='Histogram',
#                 histtype='stepfilled', alpha=0.5)
    
#     raster_arr = pct_clip(raster.read(band), pct=pct)

#     show(raster_arr)

# def show_sar(path, pct=[2,98], hist=False, meta=False):

#     raster = rasterio.open(path)
#     if meta:
#         print(raster.meta)
#     if hist:
#         show_hist(source=raster, bins=50, title='Histogram',
#                 histtype='stepfilled', alpha=0.5)
    
#     raster_arr_1 = pct_clip(raster.read(1), pct=pct)
#     raster_arr_2 = pct_clip(raster.read(2), pct=pct)

#     r = raster_arr_2
#     g = raster_arr_1
#     b = raster_arr_2 / raster_arr_1

#     show(np.array([r,g,b]))

In [ ]:
df_s1= pd.DataFrame(s1_imgs, columns=['s1_imgs'])
df_s1['s1_dates'] = pd.to_datetime([i.split("_")[4] for i in s1_imgs], format='%Y%m%dT%H%M%S')

df_s2= pd.DataFrame(s2_imgs, columns=['s2_imgs'])
df_s2['s2_dates'] = pd.to_datetime([i.split("_")[2] for i in s2_imgs], format='%Y%m%dT%H%M%S')

df_s2

In [ ]:
def findNmerge(dt):

    df_closest = df_s1.iloc[(df_s1['s1_dates']-dt).abs().argsort()]
    # display(df_closest)
    k = -1
    sar_arr = np.zeros((2, 10240, 10240), dtype=float)
    value_percent = 1.0

    while value_percent > 0.05:
        k += 1
        sar_arr = np.where(sar_arr == 0, sar_arr + rasterio.open(df_closest.iloc[k]['s1_imgs']).read(), sar_arr)
        # show(pct_clip(img_arr[0]))
        value_count = np.count_nonzero(sar_arr[0]==0.0)
        value_percent = value_count / sar_arr[0].size
        print(k+1, round(100-value_percent*100,1),'%', df_closest.iloc[k]['s1_dates']-dt)

    with open(f'Sentinel-1_Sentinel-2/result_{year}.csv', 'a') as f:
        w = csv.writer(f)
        w.writerow([k+1, f"{round(100-value_percent*100,1)}%", df_closest.iloc[k]['s1_dates']-dt])
        f.close()
    
    
    # r = pct_clip(sar_arr[1])
    # g = pct_clip(sar_arr[0])
    # b = pct_clip(sar_arr[1]) / pct_clip(sar_arr[0])
    # show(np.array([r,g,b]))
    
    return sar_arr

def normalize(x):
    return (x-np.amin(x))/(np.max(x)-np.min(x)) * 1000

In [ ]:
with open(f'Sentinel-1_Sentinel-2/result_{year}.csv', 'w', newline='') as outcsv:
    writer = csv.DictWriter(outcsv, fieldnames = ["count", "pct_coverage", "time_delta"])
    writer.writeheader()
    outcsv.close()

for i in range(len(df_s2)):
    print(i+1,"/", len(df_s2))
    img, dt = df_s2.iloc[i]['s2_imgs'], df_s2.iloc[i]['s2_dates']
    # print(Path(img).stem)
    msi = rasterio.open(img)
    msi_arr = msi.read()
    sar_arr = findNmerge(dt)

    geo_info = msi.meta
    geo_info.update({
        "count": geo_info['count'] -1 +2,
        "dtype": "float32"
        })
    
    out_fn = Path(img).stem + "_sar.tiff"
    
    # with rasterio.open(os.path.join(filserver, "Sentinel-1_Sentinel-2", str(year), out_fn), 'w', **geo_info) as dest:
    #     for j in tqdm(range(geo_info['count'])):
    #         if j < 12:
    #             band = normalize(msi_arr[j])
    #             dest.write(band,j+1)
    #         elif j >= 12:
    #             band = normalize(sar_arr[j-12])
    #             dest.write(band,j+1)
    print(out_fn)
